# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [6]:

import warnings
import sqlite3
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [ ]:
import sqlite3
sales_conn = sqlite3.connect("/Users/shahzeb/Documents/CODE/PYTHON/project/data/raw/go_sales_train.sqlite")
sales_conn

<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [8]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
sales_cur = sales_conn.cursor()

sales_cur.execute(sql_query)

tables = sales_cur.fetchall()

for table in tables:
    print(table[0])

country
order_details
order_header
order_method
product
product_line
product_type
retailer_site
return_reason
returned_item
sales_branch
sales_staff


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [9]:
import pandas as pd

sql_query2 = "SELECT * FROM sales_staff"

sales_staff = pd.read_sql(sql_query2, sales_conn)

sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [10]:
sales_staff['DATE_HIRED'] = pd.to_datetime(sales_staff['DATE_HIRED'])
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Als we hier het jaar uit willen halen, schrijven we:

In [11]:
pd.DatetimeIndex(sales_staff['DATE_HIRED']).quarter

Index([4, 2, 2, 3, 1, 2, 1, 1, 3, 3,
       ...
       3, 1, 4, 1, 3, 4, 4, 4, 4, 1],
      dtype='int32', name='DATE_HIRED', length=102)

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [12]:
sales_staff_code = sales_staff["SALES_STAFF_CODE"]
work_phone = sales_staff["WORK_PHONE"]
extension = sales_staff["EXTENSION"]
fax = sales_staff["FAX"]
email = sales_staff["EMAIL"]

Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [24]:
contact_details = pd.concat([sales_staff_code, work_phone, extension, fax, email], axis=1)

contact_details

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com
...,...,...,...,...,...
97,120,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com
98,121,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com
99,122,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com
100,123,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com


## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [25]:
df = pd.DataFrame(contact_details.head(5))

df

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [26]:
first_lang_list = ["EN"] * 5

first_lang_series = pd.Series(first_lang_list, name="FIRST_LANGUAGE")

first_lang_list

df = pd.concat([df, first_lang_series], axis=1)

df

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN


Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [27]:
second_lang_dict = {
    0: "FR",
    1: "FR",
    2: "FR",
    3: "DE",
    4: "DE"
}

second_lang_serie = pd.Series(second_lang_dict, name="SECOND_LANGUAGE")

df = pd.concat([df, second_lang_serie], axis=1)

df

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE,SECOND_LANGUAGE
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN,FR
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN,FR
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN,FR
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN,DE
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN,DE


Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [28]:
third_lang_dict = {
    "THIRD_LANGUAGE": ["NL", "NL", "JPN", "JPN", "KOR"]
}

third_lang_df = pd.DataFrame(third_lang_dict)

df = pd.concat([df, third_lang_df], axis=1)

df

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE,SECOND_LANGUAGE,THIRD_LANGUAGE
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN,FR,NL
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN,FR,NL
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN,FR,JPN
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN,DE,JPN
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN,DE,KOR


## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [31]:
new_sales_staff = {
    "SALES_STAFF_CODE": [900],
    "WORK_PHONE": ["+33 1 68 94 52 20"],
    "EXTENSION": [400.0],
    "FAX": ["+33 1 68 94 56 60"],
    "EMAIL": ["KanyeWest@grtd123.com"],
    "FIRST_LANGUAGE": ["EN"],
    "SECOND_LANGUAGE": ["FR"],
    "THIRD_LANGUAGE": ["JPN"]
}

new_sales_staff_df = pd.DataFrame

sales_staff_new = pd.concat([sales_staff, pd.DataFrame(new_sales_staff)], ignore_index=True)

sales_staff_new

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FIRST_LANGUAGE,SECOND_LANGUAGE,THIRD_LANGUAGE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6.0,NaN,NaN,NaN
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6.0,NaN,NaN,NaN
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6.0,NaN,NaN,NaN
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6.0,NaN,NaN,NaN
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38.0,NaN,NaN,NaN
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38.0,NaN,NaN,NaN
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39.0,NaN,NaN,NaN
101,124,Sabine,Grüner,Level 3 Sales Representative,+(43) 13 79 56 32,348.0,+(43) 13 79 56 33,SGruner@grtd123.com,1998-02-18,39.0,NaN,NaN,NaN


### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [33]:
sales_staff_new["FULL_NAME"] = sales_staff_new["FIRST_NAME"] + " " + sales_staff_new["LAST_NAME"]

sales_staff_new

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FIRST_LANGUAGE,SECOND_LANGUAGE,THIRD_LANGUAGE,FULL_NAME
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6.0,NaN,NaN,NaN,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6.0,NaN,NaN,NaN,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6.0,NaN,NaN,NaN,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6.0,NaN,NaN,NaN,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13.0,NaN,NaN,NaN,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38.0,NaN,NaN,NaN,François De Crée
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38.0,NaN,NaN,NaN,Yvette Lattrez
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39.0,NaN,NaN,NaN,Willi Seefelder
101,124,Sabine,Grüner,Level 3 Sales Representative,+(43) 13 79 56 32,348.0,+(43) 13 79 56 33,SGruner@grtd123.com,1998-02-18,39.0,NaN,NaN,NaN,Sabine Grüner


## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [34]:
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [32]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [35]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [37]:
sales_staff_new2 = sales_staff

sales_staff_new2["EXTENSION"] += 1

sales_staff_new2

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1643.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1634.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,326.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [39]:
sales_staff_new2 = sales_staff_new2.replace("Branch Manager", "General Manager")

sales_staff_new2

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1643.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1634.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,326.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [40]:
sales_staff_new2 = sales_staff_new2.rename(columns={"POSITION_EN" : "POSITION"})

sales_staff_new2

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1643.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1634.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,326.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [41]:
sales_staff_new2 = sales_staff_new2.drop([99, 100, 101], axis=0)

sales_staff_new2

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
94,117,Frank,Jever,Level 3 Sales Representative,+(41) 17 12 13 14,1155.0,+(41) 17 12 13 20,FJever@grtd123.com,1999-10-18,37
95,118,Gianni,Vertemati,Level 1 Sales Representative,+(41) 17 12 13 14,1255.0,+(41) 17 12 13 20,GVertemati@grtd123.com,2000-02-10,37
96,119,Gracy,Gellens,General Manager,+32 16 20.73.21,1350.0,+32 16 20.73.32,GGellens@grtd123.com,1999-09-10,38
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38


### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [42]:
sales_staff_new2 = sales_staff_new2.drop("FAX", axis=1)
sales_staff_new2

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,326.0,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...
94,117,Frank,Jever,Level 3 Sales Representative,+(41) 17 12 13 14,1155.0,FJever@grtd123.com,1999-10-18,37
95,118,Gianni,Vertemati,Level 1 Sales Representative,+(41) 17 12 13 14,1255.0,GVertemati@grtd123.com,2000-02-10,37
96,119,Gracy,Gellens,General Manager,+32 16 20.73.21,1350.0,GGellens@grtd123.com,1999-09-10,38
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,GLaermans@grtd123.com,1999-11-15,38
